In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for (dirname, _, filenames) in os.walk('./kaggle/input/'):
    for filename in filenames:
        #print(filename)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/.DS_Store
./kaggle/input/tweet-sentiment-extraction/test.csv
./kaggle/input/tweet-sentiment-extraction/train.csv
./kaggle/input/tweet-sentiment-extraction/sample_submission.csv
./kaggle/input/glove840b300dtxt/glove.840B.300d.txt
./kaggle/input/twitter-sentiment-analysis/twitter_validation.csv
./kaggle/input/twitter-sentiment-analysis/twitter_training.csv
./kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt
./kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
./kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt


In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import re
import string
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
test_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv', header=None)
train_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv', header=None)
test_df.reset_index(drop=True,inplace=True)
train_df.reset_index(drop=True,inplace=True)
df = pd.concat([train_df,test_df], axis=0)
df.drop([0], axis=1, inplace=True)
df.columns = ['platform','sentiment','text']
df.drop(['platform'], axis=1, inplace=True)
df.sentiment = df.sentiment.map({"Neutral":0, "Irrelevant":0 ,"Positive":1,"Negative":2})
df.dropna(inplace=True)

In [ ]:
df

In [ ]:
stop_words = set(stopwords.words('english'))
def data_preprocessing(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text) 
    text = ''.join([c for c in text if c not in string.punctuation])
    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)
    return text

df['text'] = df['text'].astype(str).apply(data_preprocessing)

df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.sentiment, test_size=0.2, random_state=1)

In [ ]:
y_train = y_train.astype('int')
y_test= y_test.astype('int')

In [ ]:
print(X_train.shape)
print(X_test.shape)

## TF-IDF

In [ ]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

tfv.fit(list(X_train) + list(X_test))
xtrain_tfv =  tfv.transform(X_train) 
xvalid_tfv = tfv.transform(X_test)



## Count Vectorizer

In [ ]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3))


ctv.fit(list(X_train) + list(X_test))
xtrain_ctv =  ctv.transform(X_train) 
xvalid_ctv = ctv.transform(X_test)

## GloVe

In [ ]:
glove_path = '/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'

In [ ]:
def load_word_embeddings(file=glove_path):
    embeddings={}
    with open(file,'r') as infile:
        for line in infile:
            values=line.split()
            embeddings[values[0]]=np.asarray(values[1:],dtype='float32')
    return embeddings

In [ ]:
glove_embeddings = load_word_embeddings()

In [ ]:
def sentence_features_v2(s, embeddings=glove_embeddings,emb_size=200):
    # ignore stop words
    words=s
    words=[w for w in words if w.isalpha() and w in embeddings]
    if len(words)==0:
        return np.hstack([np.zeros(emb_size)])
    M=np.array([embeddings[w] for w in words])
    return M.mean(axis=0)

In [ ]:
train_glove = np.array([sentence_features_v2(x) for x in X_train])
test_glove = np.array([sentence_features_v2(x) for x in X_test])

In [ ]:
train_glove.shape

In [ ]:
X_train.shape

## Logistic Regression

In [ ]:
clf = LogisticRegression(random_state=0, multi_class='multinomial')
clf.fit(xtrain_tfv, y_train)

In [ ]:
val_pred = clf.predict(xvalid_tfv)
print("The accurary of logistic regression with tf-idf embedding is",accuracy_score(list(y_test), val_pred))

In [ ]:
clf = LogisticRegression(random_state=0, multi_class='multinomial')
clf.fit(xtrain_ctv, y_train)
val_pred = clf.predict(xvalid_ctv)
print("The accurary of logistic regression with count-vectorizer embedding is",accuracy_score(list(y_test), val_pred))

In [ ]:
predict=pd.DataFrame(val_pred)


In [ ]:
predict_file=predict.to_csv("submission.csv", index=False)

## SVM

## XGBoost